In [1]:
import numpy as np
import torch
import torch.nn as nn

from lamp import *

In [2]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    # [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    # [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]

# data shape
batch, chan_in, length = np.array(datax).shape

# conv shape
k_size = 3
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)
np_input = np.array(datax).reshape(batch, length, chan_in)
th_input = torch.Tensor(datax)

In [3]:
conv = Conv1D(k_size, chan_in, chan_out)
output = conv.forward(np_input)
print(output.shape)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

(2, 6, 2)


array([[[-0.004 , -0.0384, -0.0396, -0.043 , -0.0081, -0.0276],
        [-0.0226, -0.0208, -0.0177,  0.0167, -0.0185, -0.0235]],

       [[-0.0358, -0.0914, -0.0616, -0.0665, -0.0291, -0.0222],
        [-0.0349, -0.0701, -0.0501, -0.099 , -0.0476, -0.0362]]])

In [4]:
conv_ = nn.Conv1d(chan_in, chan_out, k_size)
output = conv_(th_input)
output.shape

torch.Size([2, 2, 6])

In [19]:
maxp = MaxPool1D(k_size, stride=1)
output = maxp.forward(np_input)
t = output.reshape(batch, chan_in, -1)
print(maxp.indx)

maxp.backward_delta(np_input, t)


tupl 1
[1]
[1]
tupl 0
[0]
[1, 0]
tupl 1
[1]
[1, 0, 1]
tupl 0
[0]
[1, 0, 1, 0]
tupl 2
[0, 2]
[1, 0, 1, 0, 0, 2]
tupl 2
[0, 2]
[1, 0, 1, 0, 0, 2, 0, 2]
[1 0 1 0 0 2 0 2]


array([[[0.5443, 0.1517, 0.5443, 0.1517, 0.1664, 0.1664],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.5443, 0.1517, 0.5443, 0.1517, 0.1664, 0.1664],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [1.0886, 0.3034, 1.0886, 0.3034, 0.3328, 0.3328],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [1.0886, 0.3034, 1.0886, 0.3034, 0.3328, 0.3328]],

       [[0.8415, 0.6482, 0.8415, 0.6482, 0.3437, 0.3437],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.8415, 0.6482, 0.8415, 0.6482, 0.3437, 0.3437],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [1.683 , 1.2964, 1.683 , 1.2964, 0.6874, 0.6874],
        [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [1.683 , 1.2964, 1.683 , 1.2964, 0.6874, 0.6874]]])

In [6]:
maxp_ = nn.MaxPool1d(k_size, stride=1)
output = maxp_(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]]])

In [7]:
stride = 3
d_out = (length - k_size) // stride + 1
res = np.zeros((batch, d_out, chan_in))

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    res[:, k, :] = np.amax(np_input[:, t1:t2, :], axis=1)

res.reshape(batch, chan_in, -1)

array([[[0.5443, 0.2968]],

       [[0.8415, 0.8318]]])

In [8]:
res = np.zeros((batch, d_out, chan_in))
print(np_input)

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    amax = np.amax(np_input[:, t1:t2, :], axis=1).flatten()
    # print(amax)
    # print(np.where(np_input == amax, 1, 0))
    idx = np.where(np.all(np_input == amax, axis=-1))[0]
    print(idx)
    # res[:, k, :] = np_input[:, idx:]

# res.reshape(batch, chan_in, -1)

[[[0.1517]
  [0.5443]
  [0.1664]
  [0.2968]
  [0.1047]
  [0.2303]
  [0.7809]
  [0.8742]]

 [[0.6482]
  [0.8415]
  [0.3437]
  [0.5621]
  [0.8318]
  [0.5978]
  [0.2179]
  [0.5826]]]
[]
[]


In [9]:
input = maxp_(th_input)

m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([2, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]]))

In [10]:
input = maxp.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape




tupl 1
[1]
[1]
tupl 0
[0]
[1, 0]
tupl 1
[1]
[1, 0, 1]
tupl 0
[0]
[1, 0, 1, 0]
tupl 2
[0, 2]
[1, 0, 1, 0, 0, 2]
tupl 2
[0, 2]
[1, 0, 1, 0, 0, 2, 0, 2]


((2, 6),
 array([[0.5443, 0.1517, 0.5443, 0.1517, 0.1664, 0.1664],
        [0.8415, 0.6482, 0.8415, 0.6482, 0.3437, 0.3437]]))

In [11]:
a = np.ones((4, 3, 2))
print(a)
a = np.insert(a,[0],0 ,axis= 1)

print(a)
print(a.shape)

for i in range(0, 8):
    if i%2 != 0 :   
        a = np.insert(a,i,0 ,axis= 1)
print(a.shape)        


[[[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]]
[[[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]]
(4, 4, 2)
(4, 8, 2)


In [12]:
a = np.arange(10)

a[:]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
#update gradient conv
stride = 2
m1 = Conv1D(k_size, chan_in, chan_out, stride=stride)
delta = m1.forward(np_input)

d_out = (length - k_size) // stride + 1
print(d_out)
res = np.zeros((k_size, chan_in, chan_out))

for k in range(k_size):
    print(k)
    t1, t2 = k * stride, d_out + k * stride 
    print(t1, t2)
    print(np_input[:, t1:t2, :, np.newaxis])
    res[k,: , :] = np.sum(
            np_input[:, t1:t2, :, np.newaxis]
            * delta[:, :,np.newaxis ,:],
            axis=(0,1),
        )


3
0
0 3
[[[[0.1517]]

  [[0.5443]]

  [[0.1664]]]


 [[[0.6482]]

  [[0.8415]]

  [[0.3437]]]]
1
2 5
[[[[0.1664]]

  [[0.2968]]

  [[0.1047]]]


 [[[0.3437]]

  [[0.5621]]

  [[0.8318]]]]
2
4 7
[[[[0.1047]]

  [[0.2303]]

  [[0.7809]]]


 [[[0.8318]]

  [[0.5978]]

  [[0.2179]]]]


In [14]:
stride = 2
m1 = Conv1D(k_size, chan_in, chan_out, stride=stride)
delta = m1.forward(np_input)



print(delta.shape)
d_out = length

res = np.zeros((batch, length, chan_in))

if stride>1:
    for i in range(0,m1.sizeOut +1):
        if i%stride != 1 :
            delta = np.insert(delta, i, 0, axis= 1)
print(delta.shape)
for kd, kr in enumerate(list(range(0, d_out, stride))):
    print("kd", kd, " kr", kr)
    t1, t2 = 0, k_size
    d1 = -kd - 2 + k_size
    d2 = -kd - 2

    if kd == 0:
        t1, t2 = 0, kd + 1
        d1, d2 = None, -1

    elif kd < k_size - 1:
        print("1")
        t1, t2 = 0, kd + 1
        d1, d2 = -1, -kd - 2

    elif kd > d_out - (k_size + 1):
        print("2")
        t1, t2 = k_size - d_out + kd, k_size
        d1, d2 = -kd + 2, 0

    print(d1, d2, kr, kd)
    print(np.flip(params, axis=1)[np.newaxis, t1:t2, :, :])
    print(delta[:, d2:d1, np.newaxis, :])

    res[:, kr, :] = np.sum(
        np.flip(params, axis=1)[np.newaxis, t1:t2, :, :]
        * delta[:, d2:d1, np.newaxis, :],
        axis=(1, 3),
    )

res.shape, np_input.shape


(2, 3, 2)
(2, 5, 2)
kd 0  kr 0
None -1 0 0
[[[[0.94896853 0.26555139]]]]
[[[[ 0.05227267 -0.05545671]]]


 [[[ 0.03976593  0.0383053 ]]]]
kd 1  kr 2
1
-1 -3 2 1
[[[[0.94896853 0.26555139]]

  [[0.69401561 0.65266928]]]]
[[[[ 0.          0.        ]]

  [[ 0.00616713 -0.00112756]]]


 [[[ 0.          0.        ]]

  [[ 0.05741732 -0.04736351]]]]
kd 2  kr 4
-1 -4 4 2
[[[[9.48968532e-01 2.65551394e-01]]

  [[6.94015611e-01 6.52669276e-01]]

  [[7.69447651e-01 2.21191121e-04]]]]
[[[[ 0.00122545 -0.01284474]]

  [[ 0.          0.        ]]

  [[ 0.00616713 -0.00112756]]]


 [[[ 0.03029274  0.00788414]]

  [[ 0.          0.        ]]

  [[ 0.05741732 -0.04736351]]]]
kd 3  kr 6
-2 -5 6 3
[[[[9.48968532e-01 2.65551394e-01]]

  [[6.94015611e-01 6.52669276e-01]]

  [[7.69447651e-01 2.21191121e-04]]]]
[[[[ 0.          0.        ]]

  [[ 0.00122545 -0.01284474]]

  [[ 0.          0.        ]]]


 [[[ 0.          0.        ]]

  [[ 0.03029274  0.00788414]]

  [[ 0.          0.        ]]]]


((2, 8, 1), (2, 8, 1))

In [15]:




x, y, z = np.ogrid[:2,:4,:5]
M = 100*x + 10*y + z
M.shape
(2, 4, 5)
print(M)
inds = [(0, 0), (2, 1), (3, 4), (1, 2)]
inds_array = np.moveaxis(np.array(inds), -1, 0); inds_array

M[np.index_exp[:] + tuple([1])]  # same as M[:, [0, 2, 3, 1], [0, 1, 4, 2]]

M[:, [0, 2, 3, 1], [0, 1, 4, 2]]


[[[  0   1   2   3   4]
  [ 10  11  12  13  14]
  [ 20  21  22  23  24]
  [ 30  31  32  33  34]]

 [[100 101 102 103 104]
  [110 111 112 113 114]
  [120 121 122 123 124]
  [130 131 132 133 134]]]


array([[  0,  21,  34,  12],
       [100, 121, 134, 112]])

In [16]:
p = [1,0,2,5]

p += [2]



t = [0 ]* 4 + [1]
t

[0, 0, 0, 0, 1]